# Start Session & Connect GCS

In [1]:
!ls

credential  download_data.sh  notebooks   spark_gcs.ipynb  taxi_schema.ipynb
data	    lib		      rdds.ipynb  spark_sql.ipynb


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

In [3]:
# Configuration

credentials_location = '/home/diccode/zoomcamp/credential/axial-coyote-376114-4c1af9c983eb.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "./lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

In [4]:
#Context

sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

23/03/25 11:31:05 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 172.21.134.10 instead (on interface eth0)
23/03/25 11:31:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/03/25 11:31:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
# Builder

spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [6]:
path = 'gs://de_zoomcamp_prefect/data/green/*'
df_green = spark.read.parquet(path)

In [21]:
df_green.show()

+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|    pickup_datetime|   dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+-------------------+-------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2|2018-12-21 15:17:29|2018-12-21 15:18:57|                 N|         1|         264|         264|              5|          0.0|        3.0|  0.5|    0.5|       0.0

In [22]:
df_green.count()

7778101